In [1]:
# 型別宣告
# 此為斷言宣告, 若型別不同則回傳Error
x = 3::Int64

3

In [2]:
# 型別錯誤
x = 3::Int8

TypeError: TypeError: in typeassert, expected Int8, got Int64

In [3]:
# 複合型別
# 型別名稱由大寫開頭並以駝峰式命名(ex: OneTwoThree) 
struct Time
    hour::Int64
    minute::Int64
    second::Int64
end

In [4]:
time1 = Time(10, 5, 0)

Time(10, 5, 0)

In [5]:
# <物件>.<欄位>
time1.hour

10

In [6]:
# 用struct建構的型別是不可變的
time1.hour = 5

ErrorException: setfield! immutable struct of type Time cannot be changed

In [7]:
# 建立可變的型別
mutable struct Time2
    hour::Int64
    minute::Int64
    second::Int64
end

In [8]:
time2 = Time2(10, 5, 0)

Time2(10, 5, 0)

In [9]:
# 改變型別欄位
time2.hour = 5

5

In [10]:
# 型別錯誤
time2.hour = "Hi"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Integer}, !Matched::Ptr) where T<:Integer at pointer.jl:23
  ...

In [11]:
mutable struct Time3
    hour::Int64
    minute::Int64
    second::Int64
    Time3(h, m) = new(h, m, 0)  # 內部建構子
end

# 內部建構子是一個函式, 函式名稱必須與型別名稱相同
# new只能在內部建構子中使用, 參數與型別欄位依序對應
# 此例為: 只允許變更小時及分鐘欄位, 秒數固定為0

In [12]:
time3 = Time3(3, 4)

Time3(3, 4, 0)

In [13]:
# 須遵循內部建構子的設定
time3 = Time3(3, 4, 3)

MethodError: MethodError: no method matching Time3(::Int64, ::Int64, ::Int64)
Closest candidates are:
  Time3(::Any, ::Any) at In[11]:5

In [14]:
# 參數化型別, 此例的Int64為輸入的參數
Matrix{Int64}(undef, 8, 8)

8×8 Array{Int64,2}:
 55834574862   8589934594  115964117018  …             5  47244640266
 55834574863  90194313219   12884901914     150323855396            0
 55834574848  12884901909   12884901891     150323855397            0
 68719476749  12884901891   12884901891     150323855360            0
 55834574865  98784247811   12884901891     163208757283            0
 55834574861  98784247832  120259084291  …  150323855399            0
  4294967309  12884901891  120259084317     150323855395            0
  8589934594  12884901891   12884901891      42949672995            0

In [15]:
# 參數名稱建議大寫的單一字母
struct Box{T}
    element::T
end

In [16]:
x = Box{Char}('A')
typeof(x.element)

Char

In [17]:
mutable struct Time2
    hour::Int64
    minute::Int64
    second::Int64
end

format(time::Time2) = "$(time.hour):$(time.minute):$(time.second)"

format (generic function with 1 method)

In [18]:
time = Time2(12, 0, 0)
formated_time = format(time)

"12:0:0"

In [19]:
# 判斷是否為上午或下午的方法
function isam(time::Time2)
    if 0 <= time.hour < 12
        return true
    end
    return false
end

function ispm(time::Time2)
    if 12 <= time.hour < 24
        return true
    end
    return false
end

ispm (generic function with 1 method)

In [20]:
isam(time)

false

In [21]:
ispm(time)

true

In [22]:
# 抽象型別

# 建立抽象型別
abstract type Animal end

# 貓跟狗型別都是Animal的子型別, 而Animal則作為父型別
struct Dog <: Animal
    name::String
end
struct Cat <: Animal
    name::String
end

In [23]:
# 抽象型別不能被實體化
Animal()

MethodError: MethodError: no constructors have been defined for Animal

In [24]:
# 具體型別不能作為父型別
struct Poppy <: Dog
    name::String
end

ErrorException: invalid subtyping in definition of Poppy

In [25]:
# 抽象型別可作為子型別或父型別
abstract type Hippo <: Animal end

In [26]:
# 定義抽象型別的Method
name(a::Animal) = println("My name is $(a.name)")

name (generic function with 1 method)

In [27]:
d = Dog("小黃")
name(d)

My name is 小黃


In [28]:
c = Cat("咪咪")
name(c)

My name is 咪咪


In [29]:
# 查詢父型別
supertype(Dog)

Animal

In [30]:
# 沒有父型別, 內建為Any
supertype(Animal)

Any

In [31]:
# Any的父型別還是Any
supertype(Any)

Any

In [32]:
# 查詢子型別
subtypes(Animal)

3-element Array{Any,1}:
 Cat  
 Dog  
 Hippo

In [33]:
subtypes(Dog)

0-element Array{Type,1}

In [34]:
# 確認型別父子關係
Hippo <: Animal

true

In [35]:
# 初始化不完全

# 若想建立一型別, 且其中的欄位也要是該型別
# 一般情況, 我們難以為該欄位實體化一個該型別的物件
mutable struct SelfReferential
    obj::SelfReferential
end

SelfReferential()
# 下面提供解法

MethodError: MethodError: no method matching SelfReferential()
Closest candidates are:
  SelfReferential(!Matched::SelfReferential) at In[35]:6
  SelfReferential(!Matched::Any) at In[35]:6

In [36]:
# 自我參考
mutable struct SelfReferential2
    obj::SelfReferential2
    function SelfReferential2()
        x = new()
        x.obj = x
        return x
    end
end

sr2 = SelfReferential2()

SelfReferential2(SelfReferential2(#= circular reference @-1 =#))

In [37]:
println(sr2 === sr2.obj)   # '==='判斷是否為同一記憶體位址的物件
println(sr2 === sr2.obj.obj)

true
true


In [1]:
# 範例: LinkedList

mutable struct Node
    prev::Union{Node, Nothing}
    next::Union{Node, Nothing}
    data
end

function Base.show(io::IO, node::Node)
    print(io, "Node(")
    (node.prev != nothing) && print(io, " - ")
    print(io, "$(node.data)")
    (node.next != nothing) && print(io, " - ")
    print(io, ")")
end

mutable struct LinkedList
    head::Node
    tail::Node
    current::Node
    function LinkedList(data)
        node = Node(nothing, nothing, data)
        new(node, node, node)
    end
end

function Base.show(io::IO, ll::LinkedList)
    ll.current = ll.head
    print(io, "LinkedList(")
    while ll.current != ll.tail
        print(io, "$(ll.current.data) - ")
        ll.current = ll.current.next
    end
    print(io, "$(ll.current.data))")
end

function insert!(ll::LinkedList, node::Node)
    node.prev = ll.current
    node.next = ll.current.next
    if ll.current === ll.tail
        ll.tail = node
    else
        ll.current.next.prev = node
    end
    ll.current.next = node
    ll.current = node
    ll
end

function remove!(ll::LinkedList)
    if ll.current != ll.head
        ll.current.prev.next = ll.current.next
    else
        ll.head = ll.current.next
    end
    if ll.current != ll.tail
        ll.current.next.prev = ll.current.prev
        ll.current = ll.current.next
    else
        ll.tail = ll.current.prev
    end
    ll
end

remove! (generic function with 1 method)

In [2]:
ll = LinkedList(0)

LinkedList(0)

In [3]:
insert!(ll, Node(nothing, nothing, 1))
insert!(ll, Node(nothing, nothing, 2))
insert!(ll, Node(nothing, nothing, 3))
insert!(ll, Node(nothing, nothing, 4))

LinkedList(0 - 1 - 2 - 3 - 4)

In [4]:
remove!(ll)

LinkedList(0 - 1 - 2 - 3)

In [3]:
# 實作一個方法
f(x::Float64, y::Int64) = x^y

f (generic function with 1 method)

In [4]:
f(2.5, 3)

15.625

In [5]:
# 同樣的函數名稱, 可以有多種方法
f(x::Number, y::Number) = x^y
methods(f)

# 2 methods for generic function "f":
[1] f(x::Float64, y::Int64) in Main at In[3]:2
[2] f(x::Number, y::Number) in Main at In[5]:1

In [6]:
# 方法的模糊
# 因為2種方法都符合, 系統無法判定該用哪個
g(x::Float64, y) = 2x + y
g(x, y::Float64) = x + 2y
g(3.0, 4.0)

MethodError: MethodError: g(::Float64, ::Float64) is ambiguous. Candidates:
  g(x, y::Float64) in Main at In[6]:3
  g(x::Float64, y) in Main at In[6]:2
Possible fix, define
  g(::Float64, ::Float64)

In [7]:
# 應盡量把方法定義清楚
g(x::Float64, y) = 2x + y
g(x, y::Float64) = x + 2y
g(x::Float64, y::Float64) = 2x + 2y
g(3.0, 4.0)

14.0

In [1]:
# 參數化方法

# 試著建立判別型別是否相同的方法
same_type(x::T, y::T) where {T} = true
same_type(x, y) = false

same_type(5.2, 5.4)

# 此處利用多重分派的特性
# 若type相符則會使用第一個方法, 其他則用第二種

true

In [7]:
# 在陣列中加上元素

# 若元素型別相同則將元素加入陣列尾端
addone(v::Vector{T}, x::T) where {T} = [v..., x]
addone(v::Vector, x) = println("types are not consistent")

# v... 是將v陣列做展開

addone (generic function with 2 methods)

In [8]:
addone([1, 2, 3], 4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [9]:
addone([1, 2, 3], 4.0)

types are not consistent


In [16]:
# 限制第三個參數須為整數型別
foobar(a, b, x::T) where {T <: Integer} = (a, b, x)

foobar (generic function with 1 method)